In [4]:
from itertools import product
import numpy as np

# NAND
for inter in product([True, False], repeat=3):
    ouput = not (inter[0] or inter[1])
    print(f"{inter[0]} NAND {inter[1]} = {ouput}")

# ket
ket0 = np.array([[1], [0]], dtype=complex)
ket1 = np.array([[0], [1]], dtype=complex)
ket_plus = np.array([[1], [1]], dtype=complex) / np.sqrt(2)
ket_minus = np.array([[1], [-1]], dtype=complex) / np.sqrt(2)

# Hadamard

H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)
X = np.array([[0, 1], [1, 0]], dtype=complex) / np.sqrt(2)

print(H @ ket0)

# Quantum Not Gate
print(H @ ket0)
print(H @ ket0 == ket1)

True NAND True = False
True NAND True = False
True NAND False = False
True NAND False = False
False NAND True = False
False NAND True = False
False NAND False = True
False NAND False = True
[[0.70710678+0.j]
 [0.70710678+0.j]]
[[0.70710678+0.j]
 [0.70710678+0.j]]
[[False]
 [False]]


In [3]:
from abc import ABCMeta, abstractmethod
from contextlib import contextmanager
class Qubit(metaclass=ABCMeta):
    @abstractmethod
    def h(self): pass

    @abstractmethod
    def x(self): pass
    
    @abstractmethod
    def measure(self) -> bool: pass

    @abstractmethod
    def reset(self): pass

In [5]:
class QuantumDevice(metaclass=ABCMeta):
    @abstractmethod
    def allocate_qubit(self) -> Qubit: pass

    @abstractmethod
    def deallocate_qubit(self, qubit: Qubit): pass

    @abstractmethod
    def using_qubit(self) -> Qubit:
        qubit = self.allocate_qubit()
        try:
            yield qubit
        finally:
            qubit.reset()
            self.deallocate_qubit(qubit)

In [6]:
def qrng(device: QuantumDevice) -> bool:
    with device.using_qubit() as q:
        q.h()
        return q.measure()

In [11]:
class SimulatedQubit(Qubit):
    def __init__(self):
        self.reset()

    def h(self):
        self.state = H @ self.state
    
    def x(self):
        self.state = X @ self.state

    def measure(self) -> bool:
        pr0 = np.abs(self.state[0, 0]) ** 2
        sample = np.random.random() <= pr0
        return bool(0 if sample else 1)
    
    def reset(self):
        self.state = ket0.copy()

In [ ]:
#qkd

def prepare_classical_message(bit: bool, q: Qubit) -> None:
    if bit:
        q.x()

def prepare_classical_message_plusminus(bit: bool, q: Qubit) -> None:
    if bit:
        q.x()
    q.h()

def eve_measure(q: Qubit) -> bool:
    return q.measure()

def eve_measure_plusminus(q: Qubit) -> bool:
    q.h()
    return q.measure()

def send_classical_bit(device: QuantumDevice, bit: bool) -> None:
    with device.using_qubit() as q:
        prepare_classical_message(bit, q)
        result = eve_measure(q)
        q.reset()
        assert result == bit


def send_classical_bit_plusminus(device: QuantumDevice, bit: bool) -> None:
    with device.using_qubit() as q:
        prepare_classical_message_plusminus(bit, q)
        result = eve_measure_plusminus(q)
        assert result == bit

def send_classical_bit_wrong_basis(device: QuantumDevice, bit: bool) -> None:
    with device.using_qubit() as q:
        prepare_classical_message(bit, q)
        result = eve_measure_plusminus(q)
        assert result == bit

In [ ]:
#BB84